Mapa Interativo

Criando um visualização de mapa interativo que exiba o caminho mais curto entre dois pontos no mar usando a biblioteca Dash do Python.

In [1]:
!pip install dash dash-bootstrap-components dash-leaflet networkx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.5/222.5 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 285.5/285.5 kB 4.7 MB/s eta 0:00:00


In [2]:
import dash_bootstrap_components as dbc
import dash
from dash import dcc, html
import dash_leaflet as dl
import networkx as nx
import pandas as pd
from dash.dependencies import Input, Output

In [9]:
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

G = nx.Graph()

nodes = {
    "A": (37.7749, -122.4194),  # San Francisco
    "B": (34.0522, -118.2437),  # Los Angeles
    "C": (36.1699, -115.1398),  # Las Vegas
    "D": (32.7157, -117.1611),  # San Diego
}

for node, (lat, lon) in nodes.items():
  G.add_node(node, pos=(lat, lon))

edges = [
    ("A", "B"),
    ("B", "C"),
    ("C", "D"),
    ("A", "C"),
    ("B", "D"),
]

for edge in edges:
  G.add_edge(*edge)


# Layout do Dash
app.layout = dbc.Container(
    [
        dbc.Row(
            [
                dbc.Col(
                    [
                        dcc.Dropdown(
                            id='start-node',
                            options=[{'label': node, 'value': node} for node in nodes],
                            placeholder="Select start node",
                        ),
                        dcc.Dropdown(
                            id='end-node',
                            options=[{'label': node, 'value': node} for node in nodes],
                            placeholder="Select end node",
                        ),
                        html.Button('Find Shortest Path', id='find-path-btn', n_clicks=0),
                    ],
                    width=4,
                ),
                dbc.Col(
                    [
                        dl.Map(
                            children=[
                                dl.TileLayer(),
                                dl.LayerGroup(id="path-layer"),
                            ],
                            id="map",
                            style={'width': '100%', 'height': '500px'},
                            center=(36, -120),
                            zoom=6,
                        )
                    ],
                    width=8,
                ),
            ]
        )
    ],
    fluid=True,
)


@app.callback(
    Output('path-layer', 'children'),
    [Input('find-path-btn', 'n_clicks')],
    [dash.dependencies.State('start-node', 'value'),
     dash.dependencies.State('end-node', 'value')]
)

def update_path(n_clicks, start, end):
    if n_clicks > 0 and start and end:
        try:
            shortest_path = nx.shortest_path(G, source=start, target=end)
            latlngs = [nodes[node] for node in shortest_path]

            return [dl.Polyline(positions=latlngs, color='blue', weight=5)]
        except nx.NetworkXNoPath:
            return []
    return []

# Rodando o servidor
if __name__ == '__main__':
    app.run_server(debug=True)

<IPython.core.display.Javascript object>